# London Passenger Mode Choice

In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

#import packages
import pandas as pd
from pycmtensor.dataset import Dataset
from pycmtensor.expressions import Beta, RandomDraws
from pycmtensor.models import MNL

lpmc = pd.read_csv("../data/lpmc.dat", sep="\t")
lpmc = lpmc[lpmc["travel_year"]==2015]
ds = Dataset(df=lpmc, choice="travel_mode")
ds.split(0.8)

# Beta parameters
asc_walk = Beta("asc_walk", 0.0, None, None, 1)
asc_cycle = Beta("asc_cycle", 0.0, None, None, 0)
asc_pt = Beta("asc_pt", 0.0, None, None, 0)
asc_drive = Beta("asc_drive", 0.0, None, None, 0)
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
s_time = Beta("s_time", 0.5, None, None, 0)
b_purpose = Beta("b_purpose", 0.0, None, None, 0)
b_licence = Beta("b_licence", 0.0, None, None, 0)
b_car_own = Beta("b_car_own", 0.0, None, None, 0)

rnd_time = RandomDraws('rnd_time', 'lognormal', 100)

U_walk = asc_walk + b_time * ds["dur_walking"] # + rnd_time(s_time) * db["dur_walking"]
U_cycle = asc_cycle + b_time * ds["dur_cycling"] 
U_pt = asc_pt + b_time * (ds["dur_pt_rail"] + ds["dur_pt_bus"] + ds["dur_pt_int"]) + b_cost * ds["cost_transit"]
U_drive = asc_drive + b_time * ds["dur_driving"] + b_licence * ds["driving_license"] + b_cost * (ds["cost_driving_fuel"] + ds["cost_driving_ccharge"])  + b_purpose * ds["purpose"]

U = [U_walk, U_cycle, U_pt, U_drive]
mymodel = MNL(ds, locals(), U)

13:05:12 [INFO] n_train_samples:3986 n_valid_samples:997
13:05:13 [WARNING] s_time not in any utility functions
13:05:13 [WARNING] b_car_own not in any utility functions
13:05:13 [INFO] choice: travel_mode
13:05:13 [INFO] inputs in MNL: [purpose, driving_license, dur_walking, dur_cycling, dur_pt_rail, dur_pt_bus, dur_pt_int, dur_driving, cost_transit, cost_driving_fuel, cost_driving_ccharge]
13:05:14 [INFO] Build time = 00:00:00


In [2]:
from pycmtensor.models import train
from pycmtensor.optimizers import SGD, Adam, AdaGrad, SQNBFGS, RMSProp
from pycmtensor.scheduler import StepLR, ConstantLR

mymodel.reset_values()
train(mymodel, ds, optimizer=Adam, batch_size=0, base_learning_rate=1., convergence_threshold=0.001, max_steps=2000, lr_scheduler=ConstantLR, lr_stepLR_drop_every=20, lr_stepLR_factor=0.99, BFGS_warmup=30)

13:05:16 [INFO] Start (n=3986, epoch=0, LL=-5525.77, error=81.85%)
13:05:16 [INFO] Train (epoch=0, LL=-12606.28, error=52.66%, gnorm=2.64575e+00, 0/2000)
13:05:16 [INFO] Train (epoch=9, LL=-5108.05, error=48.35%, gnorm=6.78318e-01, 9/500)
13:05:16 [INFO] Train (epoch=12, LL=-4248.49, error=54.56%, gnorm=5.74250e-01, 12/500)
13:05:16 [INFO] Train (epoch=16, LL=-3696.61, error=40.02%, gnorm=4.86758e-01, 16/500)
13:05:16 [INFO] Train (epoch=43, LL=-3487.49, error=36.21%, gnorm=8.25816e-02, 43/500)
13:05:16 [INFO] Train (epoch=50, LL=-3446.44, error=35.51%, gnorm=1.03803e-01, 50/500)
13:05:16 [INFO] Train (epoch=83, LL=-3416.03, error=35.31%, gnorm=1.73152e-02, 83/500)
13:05:16 [INFO] Train (epoch=100, LL=-3415.05, error=35.31%, gnorm=9.89455e-03, 100/500)
13:05:16 [INFO] Train (epoch=122, LL=-3414.89, error=35.31%, gnorm=1.32773e-03, 122/500)
13:05:16 [INFO] Train (epoch=149, LL=-3414.86, error=35.41%, gnorm=9.41531e-04, 149/500)
13:05:16 [INFO] Model converged (t=0.393)
13:05:31 [INFO] B

In [3]:
pd.options.display.width = 99
print(mymodel.results.beta_statistics())
print(mymodel.results.model_statistics())
print(mymodel.results.benchmark())

              value   std err     t-test p-value rob. std err rob. t-test rob. p-value
asc_cycle -3.965836  0.081987 -48.371642     0.0     0.081858  -48.448031          0.0
asc_drive -2.968539   0.09233 -32.151441     0.0     0.094376  -31.454401          0.0
asc_pt    -1.385955  0.046645 -29.712946     0.0     0.049325   -28.09834          0.0
asc_walk        0.0         -          -       -            -           -            -
b_cost    -0.140486  0.012858 -10.925681     0.0     0.012273  -11.447003          0.0
b_licence  1.466081  0.080672  18.173272     0.0     0.085777   17.091847          0.0
b_purpose  0.292338  0.031839   9.181699     0.0     0.032477    9.001514          0.0
b_time    -4.988857  0.182257   -27.3726     0.0      0.19564  -25.500233          0.0
                                         value
Number of training samples used         3986.0
Number of validation samples used        997.0
Null. log likelihood              -5525.769323
Final log likelihood         

In [4]:
mymodel.results.model_correlation_matrix()

,asc_walk,asc_cycle,asc_pt,asc_drive,b_cost,b_time,b_purpose,b_licence
asc_walk,1.000000,-0.280101,0.135681,-0.552629,0.048806,-0.644047,0.432693,0.307922
asc_cycle,-0.280101,1.000000,-0.352942,-0.504412,0.127388,0.034619,0.253905,0.132023
asc_pt,0.135681,-0.352942,1.000000,-0.291221,-0.185575,0.244719,0.456596,0.264711
asc_drive,-0.552629,-0.504412,-0.291221,1.000000,-0.055131,0.317590,-0.773213,-0.476612
b_cost,0.048806,0.127388,-0.185575,-0.055131,1.000000,-0.017778,-0.025673,-0.055515
b_time,-0.644047,0.034619,0.244719,0.317590,-0.017778,1.000000,-0.091057,-0.143249
b_purpose,0.432693,0.253905,0.456596,-0.773213,-0.025673,-0.091057,1.000000,0.026699
b_licence,0.307922,0.132023,0.264711,-0.476612,-0.055515,-0.143249,0.026699,1.000000


In [5]:
mymodel.results.model_robust_correlation_matrix()

,asc_walk,asc_cycle,asc_pt,asc_drive,b_cost,b_time,b_purpose,b_licence
asc_walk,1.000000,-0.293871,0.117477,-0.562943,-0.000814,-0.697216,0.428137,0.326825
asc_cycle,-0.293871,1.000000,-0.336329,-0.469281,0.134867,0.066255,0.244732,0.094938
asc_pt,0.117477,-0.336329,1.000000,-0.319792,-0.091988,0.329933,0.442212,0.340376
asc_drive,-0.562943,-0.469281,-0.319792,1.000000,-0.068285,0.297494,-0.767246,-0.507462
b_cost,-0.000814,0.134867,-0.091988,-0.068285,1.000000,0.084258,0.021834,-0.088769
b_time,-0.697216,0.066255,0.329933,0.297494,0.084258,1.000000,-0.080324,-0.134044
b_purpose,0.428137,0.244732,0.442212,-0.767246,0.021834,-0.080324,1.000000,0.047370
b_licence,0.326825,0.094938,0.340376,-0.507462,-0.088769,-0.134044,0.047370,1.000000


In [6]:
# predictions
import numpy as np

print(mymodel.predict(ds, return_probabilities=True))
print(np.unique(mymodel.predict(ds), return_counts=True))

[[1.80567578e-02 3.05286975e-02 6.26281492e-01 ... 2.81683831e-01
  3.87216158e-05 1.72337572e-02]
 [2.60410609e-02 1.53577255e-02 1.89343256e-02 ... 1.44192382e-02
  5.00436906e-03 3.04698077e-02]
 [1.14326381e-01 1.28087412e-01 2.64089815e-01 ... 1.75316592e-01
  1.17122232e-02 1.82897697e-01]
 [8.41575801e-01 8.26026165e-01 9.06943682e-02 ... 5.28580339e-01
  9.83244686e-01 7.69398738e-01]]
(array([0, 2, 3], dtype=int64), array([172, 277, 548], dtype=int64))


In [16]:
for i in range(4):
	disaggregate_elas = mymodel.elasticities(ds, wrt_choice=i)
	print({key: np.mean(val) for key, val in disaggregate_elas.items()},"\n")

{'cost_driving_ccharge': 0.02792494930297881, 'cost_driving_fuel': 0.05899724934516689, 'cost_transit': 0.08807805485672658, 'driving_license': -0.5027457331547718, 'dur_cycling': 0.042118262595310665, 'dur_driving': 0.6648665078717327, 'dur_pt_bus': 0.2265343396824839, 'dur_pt_int': 0.07984930746890476, 'dur_pt_rail': 0.24670929798759528, 'dur_walking': -5.569327872800329, 'purpose': -0.38940586051010107} 

{'cost_driving_ccharge': 0.02792494930297881, 'cost_driving_fuel': 0.05899724934516689, 'cost_transit': 0.08807805485672658, 'driving_license': -0.5027457331547718, 'dur_cycling': -1.8451916670887203, 'dur_driving': 0.6648665078717327, 'dur_pt_bus': 0.2265343396824839, 'dur_pt_int': 0.07984930746890476, 'dur_pt_rail': 0.24670929798759528, 'dur_walking': 0.33477576059394343, 'purpose': -0.3894058605101011} 

{'cost_driving_ccharge': 0.02792494930297881, 'cost_driving_fuel': 0.05899724934516689, 'cost_transit': -0.14813309845649789, 'driving_license': -0.5027457331547718, 'dur_cyclin

In [18]:
mymodel.elasticities(ds, wrt_choice=1)

{'cost_driving_ccharge': array([0., 0., 0., ..., 0., 0., 0.]),
 'cost_driving_fuel': array([0.1779077 , 0.07665254, 0.02508586, ..., 0.02933367, 0.09876555,
        0.00742067]),
 'cost_transit': array([0.06303109, 0.15875646, 0.18932612, ..., 0.04971353, 0.03930586,
        0.11419528]),
 'driving_license': array([-1.23773298, -0.7475947 , -0.        , ..., -0.90034914,
        -1.30467349, -0.        ]),
 'dur_cycling': array([-3.6340882 , -2.33553568, -2.39725913, ..., -0.76680377,
        -1.56410125, -0.6005098 ]),
 'dur_driving': array([1.0471056 , 1.07976701, 0.26601209, ..., 0.41956413, 0.94588359,
        0.08616611]),
 'dur_pt_bus': array([0.28704319, 0.        , 0.        , ..., 0.16901989, 0.03877228,
        0.22829397]),
 'dur_pt_int': array([0.09948067, 0.        , 0.        , ..., 0.        , 0.0697901 ,
        0.        ]),
 'dur_pt_rail': array([0.        , 0.31320259, 0.46139709, ..., 0.        , 0.04652673,
        0.        ]),
 'dur_walking': array([0.00300486, 0